Для оглаживания используйте маленькую коллекцию texts.small.tar.bz2, НО **решения принимаются только по полной коллекции** texts.tar.bz2.

In [ ]:
import tarfile
from tqdm import tqdm

input_fn = './texts.small.tar.bz2'
#input_fn = './texts.tar.bz2'

with tarfile.open(input_fn, "r:bz2") as tar:
    for docid, tarinfo in tqdm(enumerate(tar)):
        with tar.extractfile(tarinfo) as inp:
            # text = inp.read().decode('utf8')
            pass


Метод suggest должен возвращать список строк кандидатов на подсказку для префикса. Не более 10 штук.


В базовом варианте прделагается реализовать пословные подсказки: предсказывать не более одного слова за раз, для предсказания использовать только введенное пользователем начало слова.


Пример: для префикса "jo" стоит вернуть самые частые слова начинающиеся на "jo", например `["john ", "joker ", "jojo "]`. При рассчете частоты не стоит различать строчные и заглавные буквы.


НО для префикса "john c" следует вернуть самые частые слова начинающиеся на "с" и добавить ко всем подсказкам слово "john", например `["john cycle ", "john cafe ", "john coffee "]`.


Если запрос пользователя заканчивается на пробел, значит он собирается ввести новое слово и использовать буквы введенные до пробела в качестве ограничения не стоит. Например, для префикса "john " следует вернуть самые частые слова коллекции и добавить ко всем подсказкам слово "john", например `["john first ", "john country ", "john one "]`.

Обратите внимание, что каждая подсказка заканчивается на пробел, тем самым выбрав Вашу подсказку пользователю будет затем предложено новое слово, а не продолжение старого.

In [ ]:
def suggest(prefix):
    return ['harry', 'potter', 'harry potter']

In [ ]:
import http.server
import socketserver
import json
import urllib

class DemoHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        parsed = urllib.parse.urlparse(self.path)
        if parsed.path == '/demo/suggest.json':
            qs = urllib.parse.parse_qs(parsed.query, keep_blank_values=True)
            if 'prefix' in qs and len(qs['prefix']) == 1:
                data = json.dumps(suggest(qs['prefix'][0])).encode('utf8')
                
                self.send_response(200)
                
                self.send_header('Content-type', 'application/json; charset=utf-8')
                self.end_headers()
                self.wfile.write(data) 
                return
            else:
                self.send_response(400)
                self.end_headers()
                self.wfile.write(b'bad request') 
                return
            
        if parsed.path == '/demo/search.json':            
            qs = urllib.parse.parse_qs(parsed.query, keep_blank_values=True)
            if 'query' in qs and len(qs['query']) == 1:
                result = [] if 'search' not in globals() else search(qs['query'][0])                
                data = json.dumps(result).encode('utf8')                
                self.send_response(200)
                
                self.send_header('Content-type', 'application/json; charset=utf-8')
                self.end_headers()
                self.wfile.write(data) 
                return
            else:
                self.send_response(400)
                self.end_headers()
                self.wfile.write(b'bad request') 
                return            
                
        http.server.SimpleHTTPRequestHandler.do_GET(self)


PORT = 8080
with socketserver.TCPServer(("", PORT), DemoHandler) as httpd:
    print("Open demo at http://localhost:%d/demo/demo.html" % PORT)
    httpd.serve_forever()

**КОНЕЦ ОБЯЗАТЕЛЬНОЙ ЧАСТИ НА 1 балл**

Далее идёт описание дополнительный заданий

###  1. Саджест
#### 1.1  (2 балл) Подсказка словочетаний

Выделим из коллекции "словосочения": такие минимальные последовательности слов, что ни первое ни последнее не является стоп-словом, и между ними нет знаков препинания. 

Например: `Lorem Ipsum is simply dummy text of the printing. Printing and typesetting industry.` разбивается на словосочетания `["Lorem Ipsum", "Ipsum is simply", "simply dummy", "dummy text", "text of the printing", "Printing and typesetting", "typesetting industry"]`

Скажем, что подсказка подходит префиксу, если последние введенные слова и последние буквы начинаются с данного префикса. Среди подходящих словосочетаний выдайте десятку самых частотных.

Таким образом в подсказках должны начать вплывать устойчивые биграмы, например для префикса "who is john c" следует вернуть самые частые словосочетания начинающиеся на "john с" и добавить ко всем подсказкам слова "who is ", например `["who is john cena ", "who is john cooper ", "who is john christopher "]`.

Для префикса "john " (с пробелом на конце) следует вернуть самые частые словосочетания начинающиеся на "john ", например `["john kennedy ", "john f ", "john cena "]`. 

Если подходящих словосочетаний меньше десяти - дополните подсказки обычными пословными подсказками.


#### 1.2  (0.5 балла) Подсказка раскладки

Если введенный префикс похож на запрос с неправильной раскладкой клавиатуры, предложите подсказки по исправленному запросу.

#### 1.3  (2 балл) Подсказка опечаток

Если введенных слов нет в словаре, зато есть похожие (отличающиеся небольшой перестановкой букв/заменой букв) - предложите исправление опечатки.

### 2 Поиск
#### 2.1 (0.5 балла) Кое-какой поиск (обязателен для остальных пунктов 2.*)
Реализуйте поиск по всей коллекции документов, в которых есть слова запроса, похожий на тот что был в первой домашней работе.

В качестве функции разнижование используте что-то несложное, например tf\*idf или bm25 из первой домашки.

Для того чтобы демка заработала  достаточно реализовать метод search. Пример результата, который ожидается от неё есть ниже в комментарии.


#### 2.2 (1 балл) Сниппеты
Реализуйте построение сниппета на основе запроса и текста документа. 

Рассмотрим все подпоследовательности слов документа, суммарной длинны не больше, скажем 300. Среди таких последовательностей выберите такую, которая больше всего "похожа" на запрос. В качестве простейшей "похожести" можно взять tf\*idf.

Слова из запроса, попавшие в сниппет, стоит выделить html тегом `<b>`


#### 2.3 (1 балл) Толковые сниппеты
В качестве функции "похожести" сниппета за запрос реализуйте что-нибудь более сложное, учитывающие стоят ли слова запроса рядом и стоят ли они в том же порядке, что и в запросе. Предлагается реализовать несколько таких "принаков похожести" и скомбинировать их (например линейно).

Продемонстрируйте на нескольких запросах превосходство вашей фукнции схожести над базовой.


#### 2.4.  (2 балла) Ранжирование
Для оценки качества поиска воспользуйтесь файлом ground_truth.txt.gz. В нем в каждой строке записан текст запроса и список документов в порядке убывания "релевантности". Точное значение релевантности неизвестно, известен только порядок следования документов.

Рассчитайте чиселку качества на основе ground truth. Подберите линейную комбинацию факторов "tfidf" и "tfidf по заголовку", которая улучшает качество.

#### 2.5  (1 балл) Факторы (зависит от 2.4)
Рассчитайте с десяток факторов ранжирования. За основу возьмите рузультаты предыдущих лабораторных (bm25, pagerank, etc). Среди факторов должны быть и статические и запросные и динамические.

На основе ground truth оценок подберите комбинацию признаков, которая улучшает ранжирование документов.

#### 2.6  (1 балл) Учет позиций слов в запросе (зависит от 2.4)
Придумайте и реализуйте набор признаков ранжирования, которые награждают за то, что слова в документе идут в том же порядке, что и в запросе.

Подберите формулу с учетом новых признаков. Продемонстрируйте, что качество поиска улучшилось.

In [ ]:
def search(query):
    return []
    """
    Каждый документ должен быть описан диктом:
      title - строка, заголовок документа - первая строчка в файле
      link - урл, имя файла в архиве совпадает с названием документа на сайте википедии
      snippet - html, составляется самостоятельно на основе запроса и текста документа

    Пример результата работы функции

    [{'title': 'Harry Potter',
      'link': 'https://simple.wikipedia.org/wiki/Harry_Potter',
      'snippet': '<b>Lorem Ipsum</b> is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry\'s standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. ',
     },
     {'title': 'Таня Гроттер', 
      'link': 'https://simple.wikipedia.org/wiki/Tanya_Grotter', 
      'snippet': '<b>Sed ut perspiciatis</b>, unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam eaque ipsa, quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt, explicabo.',
     }]
    """